<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/Baseline_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code obtained from ChatGPT with some adjusts and addings

Give me a python notebook that use BM25 for querying in the Glasgow CISI dataset.
--------------------------


[ChatGPT aswer](https://github.com/leonardo3108/IA368dd/blob/main/chats/Saved_Chat_GPT_1.md#give-me-a-python-notebook-that-use-bm25-for-querying-in-the-glasgow-cisi-dataset)

# Download the CISI dataset

> Before you begin, make sure you have downloaded the CISI dataset and stored it in a directory on your local machine. You can download the dataset from this link: https://ir.shef.ac.uk/cloughie/resources/cisi/cisi.tar.gz


O caminho fornecido pelo ChatGPT estava incorreto, modifiquei para https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz


In [1]:
!wget https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz

--2023-02-19 13:29:48--  https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz
Resolving ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)... 130.209.240.253
Connecting to ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)|130.209.240.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775144 (757K) [application/gzip]
Saving to: ‘cisi.tar.gz.1’

cisi.tar.gz.1       100%[===================>] 756.98K  1.49MB/s    in 0.5s    

2023-02-19 13:29:49 (1.49 MB/s) - ‘cisi.tar.gz.1’ saved [775144/775144]



In [2]:
!tar -xzvf cisi.tar.gz

CISI.ALL
CISI.BLN
CISI.QRY
CISI.REL


# Install and import needed packages

In [3]:
!pip install nltk
!pip install rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


> from bm25 import BM25Okapi

The package name ChatGPT gave was incorrect. I've got the correct from https://pypi.org/project/rank-bm25/

In [4]:
import os
import re
import json
import nltk
from rank_bm25 import BM25Okapi

# Set up the NLTK stopwords

I had to add to ChatGPT generated code the 'punkt' package from NLTK for correct tokenization.

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Define a function to preprocess the documents and queries

Into the ChatGPT code, I had to add a treatment for line breaks. It was adding words from different lines together.

In [6]:
def preprocess(text):
    # Convert the text to lowercase
    text = text.lower()

    # Substitute line breaks for space
    text = re.sub(r'\n', ' ', text)

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-z0-9 ]', '', text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords]

    # Stem the tokens
    stemmer = nltk.PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens]

    return tokens


In [7]:
tokenized_query = preprocess('''What problems and concerns are there in making up descriptive titles?
What difficulties are involved in automatically retrieving articles from
approximate titles?''')
print(tokenized_query)

['problem', 'concern', 'make', 'descript', 'titl', 'difficulti', 'involv', 'automat', 'retriev', 'articl', 'approxim', 'titl']


# Load the documents

The ChatGPT code was interpretating badly the CISI file format. The title (.T) and author (.A) sessions were ignorated, and cross references(.X) session should be ignored. 

Got help from https://www.kaggle.com/code/kerneler/starter-cisi-c3822057-1/data.

I adjusted the code.

In [8]:
docs_path = 'CISI.ALL'
docs = []
with open(docs_path, 'r') as f:
    doc_text = ''
    register = False
    for line in f:
        if line.startswith('.I'):
            if doc_text:
                docs.append(doc_text)
            doc_text = ''
            register = True
        elif line.startswith('.X'):
            register = False
        elif register and not line.startswith('.'):
            doc_text += line
    if doc_text:
        docs.append(doc_text)

In [9]:
docs[0]

"18 Editions of the Dewey Decimal Classifications\nComaromi, J.P.\n   The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.\n"

In [10]:
preprocess(docs[0])

['18',
 'edit',
 'dewey',
 'decim',
 'classif',
 'comaromi',
 'jp',
 'present',
 'studi',
 'histori',
 'dewey',
 'decim',
 'classif',
 'first',
 'edit',
 'ddc',
 'publish',
 '1876',
 'eighteenth',
 'edit',
 '1971',
 'futur',
 'edit',
 'continu',
 'appear',
 'need',
 'spite',
 'ddc',
 'long',
 'healthi',
 'life',
 'howev',
 'full',
 'stori',
 'never',
 'told',
 'biographi',
 'dewey',
 'briefli',
 'describ',
 'system',
 'first',
 'attempt',
 'provid',
 'detail',
 'histori',
 'work',
 'spur',
 'growth',
 'librarianship',
 'countri',
 'abroad']

# Preprocess the documents

In [11]:
docs = [preprocess(doc) for doc in docs]

In [12]:
len(docs), docs[:5]

(1460,
 [['18',
   'edit',
   'dewey',
   'decim',
   'classif',
   'comaromi',
   'jp',
   'present',
   'studi',
   'histori',
   'dewey',
   'decim',
   'classif',
   'first',
   'edit',
   'ddc',
   'publish',
   '1876',
   'eighteenth',
   'edit',
   '1971',
   'futur',
   'edit',
   'continu',
   'appear',
   'need',
   'spite',
   'ddc',
   'long',
   'healthi',
   'life',
   'howev',
   'full',
   'stori',
   'never',
   'told',
   'biographi',
   'dewey',
   'briefli',
   'describ',
   'system',
   'first',
   'attempt',
   'provid',
   'detail',
   'histori',
   'work',
   'spur',
   'growth',
   'librarianship',
   'countri',
   'abroad'],
  ['made',
   'technic',
   'librari',
   'slater',
   'report',
   'analysi',
   '6300',
   'act',
   '104',
   'technic',
   'librari',
   'unit',
   'kingdom',
   'librari',
   'one',
   'aspect',
   'wider',
   'pattern',
   'inform',
   'inform',
   'transfer',
   'librari',
   'restrict',
   'document',
   'take',
   'account',
   'd

# Build the BM25 index

In [13]:
bm25 = BM25Okapi(docs)

In [14]:
bm25.get_scores(tokenized_query)

array([0.        , 0.        , 0.        , ..., 1.95827862, 1.77080883,
       0.        ])

# Load the queries

In [15]:
queries_path = 'CISI.QRY'
queries = []
with open(queries_path, 'r') as f:
    query_text = ''
    for line in f:
        if line.startswith('.I'):
            if query_text:
                queries.append(query_text)
            query_text = ''
        elif line.startswith('.W'):
            query_text = line[3:]
        else:
            query_text += line
    if query_text:
        queries.append(query_text)

In [16]:
queries[0]

'What problems and concerns are there in making up descriptive titles?\nWhat difficulties are involved in automatically retrieving articles from\napproximate titles?\nWhat is the usual relevance of the content of articles to their titles?\n'

# Preprocess the queries

In [17]:
queries = [preprocess(query) for query in queries]

In [18]:
queries[0]

['problem',
 'concern',
 'make',
 'descript',
 'titl',
 'difficulti',
 'involv',
 'automat',
 'retriev',
 'articl',
 'approxim',
 'titl',
 'usual',
 'relev',
 'content',
 'articl',
 'titl']

# Perform the queries

ChatGPT code was ok here. Just checked, and altered K=100 to K=10 documents.



In [19]:
K = 10

# Perform the queries
retrieved_docs = {}
for i, query in enumerate(queries):
    doc_scores = bm25.get_scores(query)
    top_docs = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:K]
    retrieved_docs[i+1] = top_docs

In [20]:
doc_scores

array([ 7.44076952,  9.63984534,  0.3359594 , ...,  1.17052702,
        6.7964225 , 10.84566393])

In [21]:
sorted(doc_scores, reverse = True)[:K]

[41.853778859208425,
 39.52711178679075,
 34.53808349251481,
 33.42141397628546,
 31.50818598575653,
 31.47480646879566,
 31.37205446328315,
 31.182037695591223,
 31.09846049855547,
 30.929022306467875]

In [22]:
retrieved_docs[1]

[428, 721, 1298, 758, 64, 75, 37, 1420, 710, 602]

# Ingest the evaluation file

Manually made based on 
https://www.kaggle.com/code/kerneler/starter-cisi-c3822057-1/data?select=CISI.REL.

In [23]:
relevant_docs = {}
for query_id in range(1, len(queries) + 1):
    relevant_docs[query_id] = []

In [24]:
for line in open('CISI.REL'):
    query_id, document_ID = [int(id) for id in line.split()[:2]]
    relevant_docs[query_id].append(document_ID)

In [25]:
relevant_docs[1]

[28,
 35,
 38,
 42,
 43,
 52,
 65,
 76,
 86,
 150,
 189,
 192,
 193,
 195,
 215,
 269,
 291,
 320,
 429,
 465,
 466,
 482,
 483,
 510,
 524,
 541,
 576,
 582,
 589,
 603,
 650,
 680,
 711,
 722,
 726,
 783,
 813,
 820,
 868,
 869,
 894,
 1162,
 1164,
 1195,
 1196,
 1281]

# Print the top 10 results for each query

In [26]:
for query_id, doc_ids in retrieved_docs.items():
    print('Query', query_id)
    for i, doc_id in enumerate(doc_ids[:10]):
        print('\tRank {}: Doc {} - '.format(i+1, doc_id), '****R****' if doc_id in relevant_docs[query_id] else 'NR')

Query 1
	Rank 1: Doc 428 -  NR
	Rank 2: Doc 721 -  NR
	Rank 3: Doc 1298 -  NR
	Rank 4: Doc 758 -  NR
	Rank 5: Doc 64 -  NR
	Rank 6: Doc 75 -  NR
	Rank 7: Doc 37 -  NR
	Rank 8: Doc 1420 -  NR
	Rank 9: Doc 710 -  NR
	Rank 10: Doc 602 -  NR
Query 2
	Rank 1: Doc 1137 -  NR
	Rank 2: Doc 308 -  NR
	Rank 3: Doc 596 -  NR
	Rank 4: Doc 796 -  NR
	Rank 5: Doc 789 -  NR
	Rank 6: Doc 487 -  NR
	Rank 7: Doc 419 -  NR
	Rank 8: Doc 1070 -  NR
	Rank 9: Doc 450 -  NR
	Rank 10: Doc 1326 -  NR
Query 3
	Rank 1: Doc 1180 -  NR
	Rank 2: Doc 539 -  NR
	Rank 3: Doc 468 -  NR
	Rank 4: Doc 1178 -  NR
	Rank 5: Doc 1132 -  NR
	Rank 6: Doc 1076 -  NR
	Rank 7: Doc 1234 -  NR
	Rank 8: Doc 1276 -  NR
	Rank 9: Doc 913 -  NR
	Rank 10: Doc 1141 -  NR
Query 4
	Rank 1: Doc 600 -  NR
	Rank 2: Doc 419 -  NR
	Rank 3: Doc 76 -  NR
	Rank 4: Doc 745 -  NR
	Rank 5: Doc 178 -  NR
	Rank 6: Doc 564 -  NR
	Rank 7: Doc 174 -  NR
	Rank 8: Doc 789 -  NR
	Rank 9: Doc 71 -  NR
	Rank 10: Doc 1293 -  NR
Query 5
	Rank 1: Doc 559 -  NR
	Rank

# Evaluate

This part was obtained and adapted from another question to GPT:
**Give me a function to calculate precision, recall and F1 from a BM25 system**

[ChatGPT aswer](https://github.com/leonardo3108/IA368dd/blob/main/chats/Saved_Chat_GPT_4.md#just-give-me-a-function-to-calculate-precision-recall-and-f1-from-a-bm25-system)

In [27]:
len(queries), len(retrieved_docs), len(relevant_docs)

(112, 112, 112)

In [28]:
# Initialize counters
true_positives = 0
false_positives = 0
false_negatives = 0

# Loop over queries and compute precision and recall
for query_id in range(1, len(queries) + 1):
    true_positives += len(set(relevant_docs[query_id]).intersection(retrieved_docs[query_id]))
    false_positives += len(set(retrieved_docs[query_id]).difference(relevant_docs[query_id]))
    false_negatives += len(set(relevant_docs[query_id]).difference(retrieved_docs[query_id]))
  

In [29]:
true_positives, false_positives, false_negatives

(48, 1072, 3066)

In [30]:
# Compute precision, recall and F1
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1 = 2 * precision * recall / (precision + recall)

In [31]:
precision, recall, f1

(0.04285714285714286, 0.015414258188824663, 0.02267359470949457)

# Conclusion

For this dataset, BM25 with some preprocessing, we obtained:
*  Precision (@10 documents): 4,3%
*  Recall (@10 documents): 1,5%
*  F1 (@10 documents): 2,3%

Increasing the number of documents retrieved (K = 100, for example), we increase the recall, and then F1.

